<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong><a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<strong>To get PostgreSQL long-running queries using unSkript actions.</strong></div>
</center><center>
<h2 id="Display-PostgreSQL-Long-Running-Queries">Display PostgreSQL Long Running Queries<a class="jp-InternalAnchorLink" href="#Display-PostgreSQL-Long-Running-Queries" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1. Long Running PostgreSQL Queries<br>2. Post Slack Message<code>
</code></p>

<h3 id="Long-Running-PostgreSQL-Queries">Long Running PostgreSQL Queries<a class="jp-InternalAnchorLink" href="#Long-Running-PostgreSQL-Queries" target="_self">&para;</a></h3>
<p>Here we will use unSkript <strong>Long Running PostgreSQL Queries</strong> action. This action finds out all the long-running queries on the PostgreSQL database.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>interval</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>postgresql_queries</code></p>
</blockquote>

In [10]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List, Any, Optional, Tuple
from tabulate import tabulate
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def postgresql_long_running_queries_printer(output):
    if output is None:
        return
    
    pprint.pprint(output)


@beartype
def postgresql_long_running_queries(handle, interval: int = 5) -> Tuple:
    """postgresql_long_running_queries Runs postgres query with the provided parameters.

          :type handle: object
          :param handle: Object returned from task.validate(...).

          :type interval: int
          :param interval: Interval(in seconds).

          :rtype: All the results of the query.
      """
    # Input param validation.

    # Multi-line will create an issue when we package the Legos.
    # Hence concatinating it into a single line.
    query = "SELECT pid, user, pg_stat_activity.query_start, now() - pg_stat_activity.query_start AS query_time, query, state " \
        " FROM pg_stat_activity WHERE state = 'active' AND (now() - pg_stat_activity.query_start) > interval '%d minutes';" % interval

    cur = handle.cursor()
    cur.execute(query)
    output = []
    res = cur.fetchall()
    data = []
    for records in res:
        result = {
            "pid": records[0],
            "user": records[1],
            "query_start": records[2],
            "query_time": records[3],
            "query": records[4],
            "state": records[5]
        }
        output.append(result)
        data.append([records[0], records[4], records[5], records[3]])

    if len(res) > 0:
        headers = ["pid", "query", "state", "duration"]
        print("\n")
        output = tabulate(data, headers=headers, tablefmt="grid")

    handle.commit()
    cur.close()
    handle.close()
    if len(output) != 0:
        return (False, output)
    else:
        return (True, None)


task = Task(Workflow())

task.configure(inputParamsJson='''{
    "interval": "int(interval)"
    }''')
task.configure(outputName="postgresql_queries")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(postgresql_long_running_queries, lego_printer=postgresql_long_running_queries_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we modify the output from step 1 and return a list of dictionary items for all the long-running queries on the PostgreSQL database.</p>
<blockquote>
<p><strong>Output variable: </strong>postgresql_queries</p>
</blockquote>

In [12]:
sql_queries = []
if postgresql_queries[0] == False:
    for queries in postgresql_queries[1]:
        sql_queries.append(queries)

<h3 id="Post-Slack-Message">Post Slack Message<a class="jp-InternalAnchorLink" href="#Post-Slack-Message" target="_self">&para;</a></h3>
<p>Here we will use unSkript <strong>Post Slack Message</strong> action. This action posts the message to the slack channel about the long-running queries on the PostgreSQL database.</p>
<blockquote>
<p><strong>Input parameters:</strong> <code>channel,&nbsp;message</code></p>
</blockquote>
<blockquote>
<p><strong>Output variable:</strong> <code>message_status</code></p>
</blockquote>

In [7]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Long Running Queries : {sql_queries}\\""
    }''')
task.configure(outputName="message_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's PostgreSQL legos to run PostgreSQL Query and display and collect the long-running queries from a database and send the message to a slack channel. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io">https://us.app.unskript.io</a></p>